<a href="https://colab.research.google.com/github/130ndim/TTS_HW/blob/master/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [19]:
!rm -rf TTS_HW

In [20]:
!git clone https://github.com/130ndim/TTS_HW.git
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz


Cloning into 'TTS_HW'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 296 (delta 11), reused 17 (delta 8), pack-reused 273
Receiving objects: 100% (296/296), 4.74 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (80/80), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0   9485      0 --:--:-- --:--:-- --:--:--  9485
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3288    0  3288    0     0  21350      0 --:--:-- --:--:-- --:--:-- 21490
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   4387

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [2]:
try:
  import apex
except Exception:
    ! git clone https://github.com/NVIDIA/apex.git
    % cd apex
    ! git checkout aa756cec4359aff3df1d9abb68dc6e6e92920e0c
    ! pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
    %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8775, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 8775 (delta 1), reused 2 (delta 0), pack-reused 8765
Receiving objects: 100% (8775/8775), 14.44 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (5980/5980), done.
/content/apex
Note: checking out 'aa756cec4359aff3df1d9abb68dc6e6e92920e0c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at aa756ce minimal bert pipeline parallel test (#1216)
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of whee

In [4]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Cloning git://github.com/NVIDIA/dllogger.git (to revision 26a0f8f1958de2c0c460925ff6102a4d2486d6cc) to /tmp/pip-install-hhtjq1xp/dllogger_01e12f6da6e6440886e330e8040b0a4c
  Running command git clone -q git://github.com/NVIDIA/dllogger.git /tmp/pip-install-hhtjq1xp/dllogger_01e12f6da6e6440886e330e8040b0a4c
  Running command git rev-parse -q --verify 'sha^26a0f8f1958de2c0c460925ff6102a4d2486d6cc'
  Running command git fetch -q git://github.com/NVIDIA/dllogger.git 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
  Running command git checkout -q 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
     |████████████████████████████████| 183 kB 5.4 MB/s 
     |████████████████████████████████| 195 kB 39.1 MB/s 
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5630 sha256=5d1b44dafaf0cd93792f3aac8cdeac3ab121b1dc4158627ef05b549a8544a800
  Stored in directory: /root/.cache/pip/wheels/69/76/a4/ae417dad079ab1cae58eb65601ec01d980c5e02915f4f1792f
  Created wheel for librosa: filename=lib

In [21]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [22]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:43: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2021-12-15 23:40:15.911685 - PARAMETER | output :  ./output
DLL 2021-12-15 23:40:15.912074 - PARAMETER | dataset_path :  libri_training_data
DLL 2021-12-15 23:40:15.912164 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2021-12-15 23:40:15.912246 - PARAMETER | pyprof :  False
DLL 2021-12-15 23:40:15.912327 - PARAMETER | epochs :  10
DLL 2021-12-15 23:40:15.912420 - PARAMETER | epochs_per_ch

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [36]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-16 00:02:00.778927 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-16 00:02:00.779014 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-16 00:02:00.779161 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-16 00:02:00.779249 - PARAMETER | save_mels :  False
DLL 2021-12-16 00:02:00.779328 - PARAMETER | cuda :  True
DLL 2021-12-16 00:02:00.779407 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-16 00:02:00.779479 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_10.pt
DLL 2021-12-16 00:02:00.779549 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-16 00:02:00.779617 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-16 00:02:00.779684 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-16 00:02:00.779750 - PARAMETER | sampling_rate :  22050
DLL 2021-12-16 00:02:00.779836 - PARAMETER | stft_hop_length :  256
DLL 2021-12-16 00:02:00.779911 - PA

In [38]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [39]:
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [40]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [50]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

evaluator = torchmetrics.MetricCollection({
    'f1': torchmetrics.F1(num_classes=3),
    'acc': torchmetrics.Accuracy(num_classes=3)
})
# f1_criterion = torchmetrics.F1(num_classes=3)
# acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer(sentences, padding=True, truncation=True, max_length=max_len, return_tensors='pt')
        self.max_len = max_len
        
    def __getitem__(self, index):
        # Start
        label = [0] + [val_to_class(float(v)) for v in self.labels[index]]
        len_ = self.tokenizer.input_ids.shape[-1]

        if len(label) > len_:
            label = label[:len_]
        if len(label) < len_:
            label += [0] * (len_ - len(label))

        ids = self.tokenizer.input_ids[index]
        mask = self.tokenizer.attention_mask[index]
        # End
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    # Start
    with open(filename, "r") as fk:
        sentence = []
        labels = []
        for line in fk:
            *_, s, l, _ = line.rsplit("\t", 4)
            sentence.append(s)
            labels.append(l)
        sentences.append(" ".join(sentence))
        label_proms.append(labels)
    # End


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)


model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        optimizer.zero_grad()
        # Start
        batch = {k: v.to(device) for k, v in batch.items()}
        loss = model(**batch).loss
        loss.backward()
        # End
        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            # Start
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            mask = batch['attention_mask']

            pred = out.logits.argmax(-1).masked_select(mask == 1)
            target = batch['labels'].masked_select(mask == 1)
            
            evaluator(pred.cpu(), target.cpu())

    res = evaluator.compute()
    print(f"Accuracy: {res['acc'].item()}")
    print(f"F1 score: {res['f1'].item()}")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

EPOCH:0


  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
loss; 0.3004013001918793: 100%|██████████| 14/14 [00:12<00:00,  1.16it/s]


Accuracy: 0.8764045238494873
F1 score: 0.8764045238494873
EPOCH:1


loss; 0.036022137850522995: 100%|██████████| 14/14 [00:12<00:00,  1.16it/s]


...

Accuracy: 0.8764045238494873
F1 score: 0.8764045238494873
EPOCH:8


loss; 0.13145031034946442: 100%|██████████| 14/14 [00:12<00:00,  1.16it/s]


Accuracy: 0.8764045238494873
F1 score: 0.8764045238494873
EPOCH:9


loss; 0.10574323683977127: 100%|██████████| 14/14 [00:12<00:00,  1.16it/s]


Accuracy: 0.8752809166908264
F1 score: 0.8752809166908264


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!